In [4]:
# !pip install gspread
# !pip install oauth2client
# !pip install google-auth
# !pip install gspread_dataframe

In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe

In [6]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('../credentials/centered-flow-368315-68e92ed40d61.json', scope)
print(creds)
client = gspread.authorize(creds)

# Abre la hoja de cálculo por su nombre o URL
spreadsheet = client.open_by_url('https://docs.google.com/spreadsheets/d/16Ucu31Cg9_v_SAUgBc5dDMTkxJNacC6d0cTi7Rwa6sg/edit?usp=sharing')

In [7]:
url = 'https://datos.gob.ar/dataset/transporte-sube---cantidad-transacciones-usos-por-fecha'

In [8]:
df_transacciones_SUBE = pd.DataFrame()

response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.select(".pkg-container .pkg-actions a")
    for link in links:
        href = link.get('href')
        if href and '/dataset/transporte' not in href:
            print(href)
            df = pd.read_csv(href)
            df_transacciones_SUBE = pd.concat([df_transacciones_SUBE, df])
else:
    print("This page could not be accessed::", response.status_code)

df_transacciones_SUBE

https://archivos-datos.transporte.gob.ar/upload/Dat_Ab_Usos/dat-ab-usos-2023.csv
https://archivos-datos.transporte.gob.ar/upload/Dat_Ab_Usos/dat-ab-usos-2022.csv
https://archivos-datos.transporte.gob.ar/upload/Dat_Ab_Usos/dat-ab-usos-2021.csv
https://archivos-datos.transporte.gob.ar/upload/Dat_Ab_Usos/dat-ab-usos-2020.csv


,DIA_TRANSPORTE,NOMBRE_EMPRESA,LINEA,AMBA,TIPO_TRANSPORTE,JURISDICCION,PROVINCIA,MUNICIPIO,CANTIDAD,DATO_PRELIMINAR
0,2023-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,1,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,61,NO
1,2023-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,2B,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,11,NO
2,2023-01-01,EMPRESA BATAN S.A.,BS_AS_LINEA 715M,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1707,NO
3,2023-01-01,COMPAÑIA DE TRANSPORTE VECINAL S.A.,BS_AS_LINEA_326,SI,COLECTIVO,PROVINCIAL,BUENOS AIRES,SN,438,NO
4,2023-01-01,EMPRESA DE TRANSPORTE PERALTA RAMOS SACI,BS_AS_LINEA_512,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1189,NO
...,...,...,...,...,...,...,...,...,...,...
356623,2020-12-31,EMPRESA CEFERINO SOCIEDAD ANONIMA,Línea G,NO,COLECTIVO,MUNICIPAL,RÍO NEGRO,VIEDMA,55,NO
356624,2020-12-31,UNION PLATENSE SRL,NORTE MUNICIPAL,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,LA PLATA,3245,NO
356625,2020-12-31,A T A C O NORTE S A C I,RES_PUERTO_TIROL_Y VIC,NO,COLECTIVO,PROVINCIAL,CHACO,SN,522,NO
356626,2020-12-31,ETACER SRL,STAFE_SANTA FE_LINEA_907,NO,COLECTIVO,NACIONAL,JN,SD,362,NO


In [9]:
def agregar_anio_mes(row):
    fecha = row['DIA_TRANSPORTE']
    anio, mes, _ = fecha.split("-")
    periodo = anio+"-"+mes
    return anio, mes, periodo

In [10]:
df_transacciones_SUBE['anio'], df_transacciones_SUBE['mes'], df_transacciones_SUBE['periodo'] = zip(*df_transacciones_SUBE.apply(agregar_anio_mes, axis=1))
df_transacciones_SUBE

,DIA_TRANSPORTE,NOMBRE_EMPRESA,LINEA,AMBA,TIPO_TRANSPORTE,JURISDICCION,PROVINCIA,MUNICIPIO,CANTIDAD,DATO_PRELIMINAR,anio,mes,periodo
0,2023-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,1,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,61,NO,2023,01,2023-01
1,2023-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,2B,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,11,NO,2023,01,2023-01
2,2023-01-01,EMPRESA BATAN S.A.,BS_AS_LINEA 715M,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1707,NO,2023,01,2023-01
3,2023-01-01,COMPAÑIA DE TRANSPORTE VECINAL S.A.,BS_AS_LINEA_326,SI,COLECTIVO,PROVINCIAL,BUENOS AIRES,SN,438,NO,2023,01,2023-01
4,2023-01-01,EMPRESA DE TRANSPORTE PERALTA RAMOS SACI,BS_AS_LINEA_512,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1189,NO,2023,01,2023-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
356623,2020-12-31,EMPRESA CEFERINO SOCIEDAD ANONIMA,Línea G,NO,COLECTIVO,MUNICIPAL,RÍO NEGRO,VIEDMA,55,NO,2020,12,2020-12
356624,2020-12-31,UNION PLATENSE SRL,NORTE MUNICIPAL,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,LA PLATA,3245,NO,2020,12,2020-12
356625,2020-12-31,A T A C O NORTE S A C I,RES_PUERTO_TIROL_Y VIC,NO,COLECTIVO,PROVINCIAL,CHACO,SN,522,NO,2020,12,2020-12
356626,2020-12-31,ETACER SRL,STAFE_SANTA FE_LINEA_907,NO,COLECTIVO,NACIONAL,JN,SD,362,NO,2020,12,2020-12


In [12]:
row=1
col=1
worksheet = spreadsheet.get_worksheet(0)
set_with_dataframe(worksheet,df_transacciones_SUBE,row,col)

APIError: {'code': 400, 'message': 'Invalid requests[0].updateSheetProperties: This action would increase the number of cells in the workbook above the limit of 10000000 cells.', 'status': 'INVALID_ARGUMENT'}